In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import time
import datetime
import statsmodels.api as sm
from collections import Counter, defaultdict
import re
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_regression
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import cross_val_predict
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scipy.interpolate import LSQUnivariateSpline
from scipy import interpolate
import patsy

2024-11-14 19:23:23.952055: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# TRAINING & Selection

### functions

In [ ]:
def dummy_generator(df, name):
    al = set(df[name])
    if '--' in al: 
        dummy_vars = pd.get_dummies(df[name], prefix=name).astype(int)
        dummy_vars = dummy_vars.drop(f'{name}_--', axis = 1)
        lis_d = list(dummy_vars.columns)
    else:
        dummy_vars = pd.get_dummies(df[name], prefix=name, drop_first = True).astype(int)
    df = df.drop(name, axis = 1)
    lis_d = list(dummy_vars.columns)
    final = pd.concat([df, dummy_vars], axis=1)
    
    return final , lis_d

def regression_DataFrame(df, names):
    d = {x: df[x] for x in names}
    return pd.DataFrame(d)

def mean_centering(row):
    for n in list(row.columns):
        arr = np.array(row[n])
        mean = np.mean(arr)
        centered = arr - mean
        row[n] = centered 
    return row

def log_(value):
    if value == '--':
        return '--'
    else: return np.log(value)

def stab(value):
    if value == '--':
        return '--'
    else: return ((value)**8)/10**16


def dum(value):
    if value > 1:
        return 1
    else: return 0

def dum_2(value):
    if value == '--':
        return 0
    else: return 1

def gen_splines(dataframe ,name, df ):
    spline_des = patsy.dmatrix(f"bs(dataframe[name], df={df}, degree=3, include_intercept=False) - 1", {"x": dataframe[name]})
    #dataframe = dataframe.drop(name, axis = 1) 
    sq_splines = pd.DataFrame(spline_des, columns = [f'SQM_spline_{x}' for x in range(df)])
    dataframe = pd.concat([dataframe.reset_index(drop=True),sq_splines.reset_index(drop=True)], axis = 1)
    return dataframe


#### Ridge regression

In [280]:

prep,_ = dummy_generator(prep, 'energy_efficiency_class')
prep,_ = dummy_generator(prep, 'conditions')
prep,_ = dummy_generator(prep, 'floor')
prep,_ = dummy_generator(prep, 'bathrooms_number')
prep,_ = dummy_generator(prep, 'lift')


In [364]:
prep= train[train['square_meters'] != '--']

prep = prep.drop(['attic','furnished',              
       'cellar', 'closet', 'doubleexposure',
        'est',
       'fulldayconcierge', 'half-dayconcierge',
          'kitchen',  'internalexposure',
        'north', 'onlykitchenfurnished', 'opticfiber',
       'partiallyfurnished','fireplace',
       'privateandsharedgarden', 'reception',
       'securitydoor', 'singletvsystem',
        'tavern', 'tenniscourt','south',
        'tvsystemwithsatellitedish','sharedgarden',
       'videoentryphone', 'west','hydromassage','electricgate','year_of_construction',
        'zone_second','1balcony','alarmsystem','alarmsystem1balcony','balcony','centralizedtvsystem','disabledaccess','privategarden','car_parking'], axis = 1)


prep['square_meters'] = prep['square_meters'].apply(log_)


prep['y'] = prep['y'].apply(log_)
prep['condominium_fees'] = prep['condominium_fees'].replace('--', 1)
prep['condominium_fees_dummy'] = prep['condominium_fees'].apply(dum)
prep['condominium_fees'] = prep['condominium_fees'].apply(log_)







prep,l_zones = dummy_generator(prep, 'zone')


INTs = []
for col in l_zones:
     INTs.append(prep[col]*prep['square_meters'])
prep = pd.concat([prep,pd.DataFrame({ col + 'INT':INTs[i]  for i,col in enumerate(l_zones) })], axis = 1)




prep['total_floors_in_building'] = prep['total_floors_in_building'].replace('--', 0)

prep['energy_efficiency_class'] = prep['energy_efficiency_class'].replace('--', 3)
 
prep[ 'conditions'] = prep[ 'conditions'].replace('--', 2)
prep[ 'floor'] = prep[ 'floor'].replace('--', 1)
prep['bathrooms_number'] = prep['bathrooms_number'].replace('--', 1)
prep['lift'] = prep['lift'].replace('--', 0)




RIDGE = prep.drop(['other_features'], axis = 1)
RIDGE = RIDGE.dropna()


In [365]:
RIDGE = RIDGE.applymap(int) 


In [366]:
moo = sm.OLS( np.asarray(RIDGE['y']), RIDGE.drop('y', axis = 1)).fit()
print(moo.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.724
Method:                 Least Squares   F-statistic:                     72.32
Date:                Wed, 15 May 2024   Prob (F-statistic):               0.00
Time:                        19:39:25   Log-Likelihood:                -3056.2
No. Observations:                7993   AIC:                             6702.
Df Residuals:                    7698   BIC:                             8763.
Df Model:                         294                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [363]:

prep= train[train['square_meters'] != '--']

prep = prep.drop([ 'disabledaccess','est','fireplace','furnished', 'hydromassage',
                  'securitydoor','singletvsystem','south','tavern','tvsystemwithsatellitedish',
                  'videoentryphone','west','year_of_construction'], axis = 1)


prep['square_meters'] = prep['square_meters'].apply(log_)


prep['y'] = prep['y'].apply(log_)


prep,l_zones = dummy_generator(prep, 'zone')
prep,l_zones_second = dummy_generator(prep, 'zone_second')

INTs = []
for col in l_zones:
     INTs.append(prep[col]*prep['square_meters'])
prep = pd.concat([prep,pd.DataFrame({ col + 'INT':INTs[i]  for i,col in enumerate(l_zones) })], axis = 1)



prep = gen_splines(prep, 'square_meters', 20)

prep = gen_splines(prep, 'rooms_number', 8)



prep['condominium_fees'] = prep['condominium_fees'].replace('--', 1)
prep['condominium_fees_dummy'] = prep['condominium_fees'].apply(dum)
prep['condominium_fees'] = prep['condominium_fees'].apply(log_)

prep['condominium_fees_dummy_INT'] = prep['condominium_fees_dummy']*prep['condominium_fees']


prep['total_floors_in_building'] = prep['total_floors_in_building'].replace('--', 0)
prep = gen_splines(prep, 'total_floors_in_building', 20)



prep,_ = dummy_generator(prep, 'energy_efficiency_class')
prep,_ = dummy_generator(prep, 'conditions')
prep,_ = dummy_generator(prep, 'floor')
prep,_ = dummy_generator(prep, 'bathrooms_number')
prep,_ = dummy_generator(prep, 'lift')



RIDGE = prep.drop(['other_features'], axis = 1)
RIDGE = RIDGE.dropna()

RIDGE_1 = RIDGE.iloc[:6000]
RIDGE_2 = RIDGE.iloc[6000:]

In [290]:
pipeline = make_pipeline(StandardScaler(), Ridge())

parameters = {'ridge__alpha': [38]}

# Perform Grid Search Cross-Validation
grid_search = GridSearchCV(pipeline, parameters, cv=10)
grid_search.fit(RIDGE_1.drop('y',axis = 1), RIDGE_1['y'])

best_params = grid_search.best_params_
best_score = -grid_search.best_score_
all_scores = grid_search.cv_results_

print("Best Parameters:", best_params)
print("Best Score:", -best_score)

# Optionally, you can retrieve the best estimator
ridge_estimator = grid_search.best_estimator_

Best Parameters: {'ridge__alpha': 38}
Best Score: 0.9096110004302203


### xgb

In [48]:
def prep_boosting(base, test = True):
    if test:
        df = base.copy()
        df['square_meters'] = df['square_meters'].replace('--', 120)
    else:
        df = base[base['square_meters'] != '--']
        
    #df = df[df['other_features'] != '--']
    
    df['square_meters'] = df['square_meters'].astype('float')

    df = df.drop('other_features', axis = 1)
    for col in list(df.columns):
        df[col] = df[col].replace('--', np.NaN)
    df['zone'] =df['zone'].astype('category')
    df['zone_second'] =df['zone_second'].astype('category')

    df['private_parking'] =df['private_parking'].astype('int')
    df['shared_parking'] =df['shared_parking'].astype('int')
    
    df = df.drop(['attic','pool','furnished',              
       'cellar', 'closet', 'doubleexposure',
        'est', 'fireplace',
       'fulldayconcierge', 'half-dayconcierge',
          'kitchen',  'internalexposure',
        'north', 'onlykitchenfurnished', 'opticfiber',
       'partiallyfurnished',
       'privateandsharedgarden', 'reception',
       'securitydoor', 'singletvsystem',
        'tavern', 'tenniscourt','south',
        'tvsystemwithsatellitedish','sharedgarden',
       'videoentryphone', 'west','hydromassage','electricgate'], axis = 1)
    

    return df

In [49]:
XGB = prep_boosting(train, False)

XGB_1 = XGB.iloc[:6000]
XGB_2 = XGB.iloc[6000:]

/var/folders/n2/fq3dzxdx7k34xryrftxsrz840000gn/T/ipykernel_3175/2375011381.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['square_meters'] = df['square_meters'].astype('float')


In [146]:
param_grid = {
    'objective': ['reg:squarederror'],
    'enable_categorical': [True],
    'random_state':[45],
    
    'max_depth': [6],
    'min_child_weight': [3],
    'subsample': [0.7],
    'colsample_bytree': [0.7],
    'learning_rate': [0.025],
    'n_estimators': [2100], # 2000
    'lambda': [0],
    'alpha': [0],
    'validate_parameters': [True],
    'tree_method': ['hist'],
    'missing': [np.NaN],
    'max_leaves': [150],
    'booster': ['gbtree'],
}


xgb_model = xgb.XGBRegressor(**param_grid)

kfold = KFold(n_splits=10, shuffle=True, random_state=88)
gb_est = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=kfold, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_result = gb_est.fit(XGB_1.drop('y',axis =1), XGB_1['y'])



best_params = grid_result.best_params_
all_scores = grid_result.cv_results_
best_score = grid_result.best_score_
print(best_params, -best_score)


{'alpha': 0, 'booster': 'gbtree', 'colsample_bytree': 0.7, 'enable_categorical': True, 'lambda': 0, 'learning_rate': 0.025, 'max_depth': 6, 'max_leaves': 150, 'min_child_weight': 3, 'missing': nan, 'n_estimators': 2100, 'objective': 'reg:squarederror', 'random_state': 45, 'subsample': 0.7, 'tree_method': 'hist', 'validate_parameters': True} 74648.9328951823


In [146]:
param_grid = {
    'objective': ['reg:squarederror'],
    'enable_categorical': [True],
    'random_state':[45],
    
    'max_depth': [6],
    'min_child_weight': [3],
    'subsample': [0.7],
    'colsample_bytree': [0.7],
    'learning_rate': [0.025],
    'n_estimators': [2100], # 2000
    'lambda': [0],
    'alpha': [0],
    'validate_parameters': [True],
    'tree_method': ['hist'],
    'missing': [np.NaN],
    'max_leaves': [150],
    'booster': ['gbtree'],
}


xgb_model = xgb.XGBRegressor(**param_grid)

kfold = KFold(n_splits=10, shuffle=True, random_state=88)
gb_est = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=kfold, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_result = gb_est.fit(XGB_1.drop('y',axis =1), XGB_1['y'])



best_params = grid_result.best_params_
all_scores = grid_result.cv_results_
best_score = grid_result.best_score_
print(best_params, -best_score)


{'alpha': 0, 'booster': 'gbtree', 'colsample_bytree': 0.7, 'enable_categorical': True, 'lambda': 0, 'learning_rate': 0.025, 'max_depth': 6, 'max_leaves': 150, 'min_child_weight': 3, 'missing': nan, 'n_estimators': 2100, 'objective': 'reg:squarederror', 'random_state': 45, 'subsample': 0.7, 'tree_method': 'hist', 'validate_parameters': True} 74648.9328951823


# Predict

#### make predictions

In [14]:

prep = test.copy()
prep['square_meters'] = prep['square_meters'].replace('--', 200) 

prep = prep.drop([ 'disabledaccess','est','fireplace','furnished', 'hydromassage',
                  'securitydoor','singletvsystem','south','tavern','tvsystemwithsatellitedish',
                  'videoentryphone','west','year_of_construction'], axis = 1)


prep['square_meters'] = prep['square_meters'].apply(log_)



prep,l_zones = dummy_generator(prep, 'zone')
prep,l_zones_second = dummy_generator(prep, 'zone_second')

INTs = []
for col in l_zones:
     INTs.append(prep[col]*prep['square_meters'])
prep = pd.concat([prep,pd.DataFrame({ col + 'INT':INTs[i]  for i,col in enumerate(l_zones) })], axis = 1)





prep = gen_splines(prep, 'square_meters', 20)

prep = gen_splines(prep, 'rooms_number', 8)



prep['condominium_fees'] = prep['condominium_fees'].replace('--', 1)
prep['condominium_fees_dummy'] = prep['condominium_fees'].apply(dum)
prep['condominium_fees'] = prep['condominium_fees'].apply(log_)

prep['condominium_fees_dummy_INT'] = prep['condominium_fees_dummy']*prep['condominium_fees']


prep['total_floors_in_building'] = prep['total_floors_in_building'].replace('--', 0)
prep = gen_splines(prep, 'total_floors_in_building', 20)



prep,_ = dummy_generator(prep, 'energy_efficiency_class')
prep,_ = dummy_generator(prep, 'conditions')
prep,_ = dummy_generator(prep, 'floor')
prep,_ = dummy_generator(prep, 'bathrooms_number')
prep,_ = dummy_generator(prep, 'lift')


XY_test_ridge = prep.drop(['other_features'], axis = 1)
XY_test_ridge = XY_test_ridge.dropna()

#### Perform stacking, the OLS coefficients are 0.7 and 0.3

In [15]:
def aver_gen(yr, yb):
    fin =  np.exp(np.log(yb)*0.7 + yr*0.3)  
    return fin

In [16]:
XY_test_boosting = prep_boosting(test)

In [17]:
final = pd.DataFrame()
final['ridge'] = ridge_estimator.predict(XY_test_ridge)
final['boosting'] = gb_est.predict(XY_test_boosting)


In [18]:
final['averaged'] = aver_gen(final['ridge'], final['boosting'])